# How might I dynamically update a connector stored in GitHub?
- retrieve the 'correct dataset'
- retrieve the stream confiugration
- get GitHub configuration
- update stream

## import credentials

- username and password stored in env file
- DomoAuth class has several classes for handling authentication

In [ ]:
# %pip install domolibrary

In [27]:
import os
import domolibrary.client.DomoAuth as dmda


auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

await auth.print_is_token()

warning this token has not been validated by who_am_i, run get_auth_token first
🎉 token_auth token retrieved from domo-community ⚙️


True

# Search DomoDatacenter for dataset
- you could just use DomoDatset.get_by_id()

## sample search criteria in DomoDatacenter
- just monitor network traffic

In [ ]:
{
    "entities": ["DATASET"],
    "count": 0,
    "offset": 0,
    "combineResults": False,
    "query": "*",
    "queryProfile": "DATA_CENTER",
    "filters": [
        {
            "filterType": "term",
            "field": "dataprovidername_facet",
            "value": "Snowflake Unload V2",
            "name": "Snowflake Unload V2",
            "not": False,
        }
    ],
    "facetValuesToInclude": None,
    "facetValueLimit": 400,
    "facetValueOffset": 0,
    "includePhonetic": True,
    "useEntities": True,
}

# search DomoDatacenter

- retrieve dataset
- retrieve stream_id
- streams handle how data is ingested into Domo

In [28]:
import domolibrary.classes.DomoDatacenter as dmdc

domo_datacenter = dmdc.DomoDatacenter(auth=auth)

additional_filters_ls = [
    {
        "filterType": "term",
        "field": "dataprovidername_facet",
        "value": "JSON",
    }
]

domo_datasets = await domo_datacenter.search_datasets(
    auth=auth, additional_filters_ls=additional_filters_ls
)
domo_datasets[0:5]

[DomoDataset(id='5f90901a-f0e6-451e-9bc9-09e23e96bd7f', display_type='json5', data_provider_type='json5', name='t1', description='', row_count=1, column_count=6, stream_id=1658, owner={'id': '1893952720', 'name': 'Jae Wilson1', 'type': 'USER', 'group': False}, formula={}, schema=DomoDataset_Schema(dataset=..., columns=[]), tags=DomoDataset_Tags(dataset=..., tag_ls=[]), certification=None, PDP=<domolibrary.classes.DomoPDP.Dataset_PDP_Policies object at 0x00000218D852B0D0>),
 DomoDataset(id='5eb2e87e-8e5a-43b6-ae17-93c5eb0cd89a', display_type='json5', data_provider_type='json5', name='DomoStats| Card Metadata', description='', row_count=3028, column_count=62, stream_id=1194, owner={'id': '68216396', 'name': 'Elliott Leonard', 'type': 'USER', 'group': False}, formula={}, schema=DomoDataset_Schema(dataset=..., columns=[]), tags=DomoDataset_Tags(dataset=..., tag_ls=[]), certification=None, PDP=<domolibrary.classes.DomoPDP.Dataset_PDP_Policies object at 0x00000218F984E4D0>),
 DomoDataset(id=

In [29]:
dataset_id = "5f90901a-f0e6-451e-9bc9-09e23e96bd7f"
domo_dataset = next((ds for ds in domo_datasets if ds.id == dataset_id))
domo_dataset.__dict__

{'auth': DomoTokenAuth(domo_instance='domo-community', token_name='token_auth', is_valid_token=True, url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true'),
 'id': '5f90901a-f0e6-451e-9bc9-09e23e96bd7f',
 'display_type': 'json5',
 'data_provider_type': 'json5',
 'name': 't1',
 'description': '',
 'row_count': 1,
 'column_count': 6,
 'stream_id': 1658,
 'owner': {'id': '1893952720',
  'name': 'Jae Wilson1',
  'type': 'USER',
  'group': False},
 'formula': {},
 'schema': DomoDataset_Schema(dataset=DomoDataset(id='5f90901a-f0e6-451e-9bc9-09e23e96bd7f', display_type='json5', data_provider_type='json5', name='t1', description='', row_count=1, column_count=6, stream_id=1658, owner={'id': '1893952720', 'name': 'Jae Wilson1', 'type': 'USER', 'group': False}, formula={}, schema=..., tags=DomoDataset_Tags(dataset=..., tag_ls=[]), certification=None, PDP=<domolibrary.classes.DomoPDP.Dataset_PDP_Policies object at 0x00000218D852B0D0>), columns=[]),
 'tags': DomoDataset

# Update Stream
- get the stream
- extract the configuration information
- update config settings with new SQL
- push updated stream config to dataset

In [30]:
import domolibrary.routes.stream as stream_routes

stream_id = domo_dataset.stream_id

res = await stream_routes.get_stream_by_id(auth=auth, stream_id=stream_id)
stream = res.response

stream

adjusting num_stacks_to_drop, consider revising `get_traceback` call
{'stack_length': 16, 'module_index': 12, 'num_stacks_to_drop_passed': 3}


{'id': 1658,
 'valid': True,
 'invalidExecutionId': None,
 'transport': {'type': 'CONNECTOR',
  'description': 'com.domo.connector.json.customparsing',
  'version': '4'},
 'updateMethod': 'REPLACE',
 'dataProvider': {'id': 3251,
  'key': 'json5',
  'name': 'JSON',
  'url': None,
  'defaultConnectorId': None,
  'defaultConnectorLabel': None,
  'authenticationScheme': 'fields',
  'connectorValidatorPresent': False,
  'resourceBundlePresent': False,
  'moduleHandler': None,
  'iconPicker': False,
  'unsafe': False,
  'scope': '0',
  'producer': 'DOMO',
  'version': '0',
  'dateCreated': 1601579548000,
  'dateUpdated': 1693235941000,
  'visibility': 'WITH_CONNECTOR',
  'type': 'STANDARD',
  'customerOAuthConfigurationDataProviderType': None,
  'authenticationSchemeConfiguration': [{'id': 12114,
    'name': 'authentication',
    'type': 'dropdown',
    'text': 'Authentication Type',
    'tooltipText': 'Authentication Type',
    'required': False,
    'encrypted': False,
    'defaultValue': 

In [31]:
stream_configuration = stream["configuration"]
stream_configuration

[{'streamId': 1658,
  'category': 'METADATA',
  'name': 'schema',
  'type': 'string',
  'value': '{"automatedSchema":true}'},
 {'streamId': 1658,
  'category': 'METADATA',
  'name': 'cloud',
  'type': 'string',
  'value': 'domo'},
 {'streamId': 1658,
  'category': 'METADATA',
  'name': 'jsonPaging',
  'type': 'string',
  'value': '{"baseUrl":"https://jsonplaceholder.typicode.com/todos/3","tokenType":"path","pagingType":"none","parsing":"","total":"true","pageLimitValue":0,"startPage":"1","encodeParameterValue":true,"encodeParameterKey":true,"pageParameter":"","tokenLocation":"first"}'},
 {'streamId': 1658,
  'category': 'METADATA',
  'name': 'jsonSelection',
  'type': 'string',
  'value': '{"connectionMethod": "Advanced", "jsonUrl": "https://jsonplaceholder.typicode.com/todos/7", "httpMethod": "GET", "jsonLineReader": "false", "escapeBackslash": "false", "httpsHeaders": [], "body": "", "queryParameters": [], "useDateFilter": false, "dateSelection": {"dateType": "date_range", "startDate

In [32]:
original_obj = None
for index, obj in enumerate(stream_configuration):
    if obj["category"] == "METADATA" and obj["name"] == "jsonSelection":
        stream_configuration.pop(index)
        original_obj = obj

print(
    f"""
original_obj {original_obj}
position in config {index}"""
)


original_obj {'streamId': 1658, 'category': 'METADATA', 'name': 'jsonSelection', 'type': 'string', 'value': '{"connectionMethod": "Advanced", "jsonUrl": "https://jsonplaceholder.typicode.com/todos/7", "httpMethod": "GET", "jsonLineReader": "false", "escapeBackslash": "false", "httpsHeaders": [], "body": "", "queryParameters": [], "useDateFilter": false, "dateSelection": {"dateType": "date_range", "startDate": {"type": "relative", "offset": 0}, "endDate": {"type": "relative", "offset": 0}}, "dateQueryParamsFormat": "yyyy-MM-dd", "certificateInputType": "NoCertificate", "encodeParameterKey": true, "encodeParameterValue": true, "datasetType": "static", "dateParameter": {"dateType": "single_date", "dateFrom": "relative", "dateFromOffset": 1, "dateTo": "relative", "dateToOffset": 0, "date": "relative", "dateOffset": 1}, "useBody": false, "dynamicValuesType": "enter", "rateLimitSelection": "false", "timeUnit": "second", "dateParameterType": "separate"}'}
position in config 8


In [33]:
# the original config object doesn't store the "value" as json, instead it's stored as a string
# convert to json for manipulation
import json

new_value = json.loads(original_obj["value"])
new_value

{'connectionMethod': 'Advanced',
 'jsonUrl': 'https://jsonplaceholder.typicode.com/todos/7',
 'httpMethod': 'GET',
 'jsonLineReader': 'false',
 'escapeBackslash': 'false',
 'httpsHeaders': [],
 'body': '',
 'queryParameters': [],
 'useDateFilter': False,
 'dateSelection': {'dateType': 'date_range',
  'startDate': {'type': 'relative', 'offset': 0},
  'endDate': {'type': 'relative', 'offset': 0}},
 'dateQueryParamsFormat': 'yyyy-MM-dd',
 'certificateInputType': 'NoCertificate',
 'encodeParameterKey': True,
 'encodeParameterValue': True,
 'datasetType': 'static',
 'dateParameter': {'dateType': 'single_date',
  'dateFrom': 'relative',
  'dateFromOffset': 1,
  'dateTo': 'relative',
  'dateToOffset': 0,
  'date': 'relative',
  'dateOffset': 1},
 'useBody': False,
 'dynamicValuesType': 'enter',
 'rateLimitSelection': 'false',
 'timeUnit': 'second',
 'dateParameterType': 'separate'}

In [34]:
# read in the new parameter value
new_query_path = "./new_query.txt"
assert os.path.exists(new_query_path)

with open(new_query_path, "r", encoding="utf-8") as f:
    new_query = f.read()

new_query

'https://jsonplaceholder.typicode.com/todos/8'

In [35]:
# update new obj with string version of new_value with updated query parameter

new_value.update({"jsonUrl": new_query})
new_obj = {**original_obj, "value": json.dumps(new_value)}

# add new_obj back to the stream configuration
stream_configuration.append(new_obj)

new_obj

{'streamId': 1658,
 'category': 'METADATA',
 'name': 'jsonSelection',
 'type': 'string',
 'value': '{"connectionMethod": "Advanced", "jsonUrl": "https://jsonplaceholder.typicode.com/todos/8", "httpMethod": "GET", "jsonLineReader": "false", "escapeBackslash": "false", "httpsHeaders": [], "body": "", "queryParameters": [], "useDateFilter": false, "dateSelection": {"dateType": "date_range", "startDate": {"type": "relative", "offset": 0}, "endDate": {"type": "relative", "offset": 0}}, "dateQueryParamsFormat": "yyyy-MM-dd", "certificateInputType": "NoCertificate", "encodeParameterKey": true, "encodeParameterValue": true, "datasetType": "static", "dateParameter": {"dateType": "single_date", "dateFrom": "relative", "dateFromOffset": 1, "dateTo": "relative", "dateToOffset": 0, "date": "relative", "dateOffset": 1}, "useBody": false, "dynamicValuesType": "enter", "rateLimitSelection": "false", "timeUnit": "second", "dateParameterType": "separate"}'}

In [ ]:
import datetime as dt

stream["description"] = f"updated at {dt.datetime.now()}"
stream["configuration"] = stream_configuration

stream

In [36]:
await stream_routes.update_stream(stream_id=stream_id, auth=auth, body=stream)

adjusting num_stacks_to_drop, consider revising `get_traceback` call
{'stack_length': 16, 'module_index': 12, 'num_stacks_to_drop_passed': 3}


ResponseGetData(status=200, response={'id': 1658, 'valid': True, 'invalidExecutionId': None, 'transport': {'type': 'CONNECTOR', 'description': 'com.domo.connector.json.customparsing', 'version': '4'}, 'updateMethod': 'REPLACE', 'dataProvider': {'id': 3251, 'key': 'json5', 'name': 'JSON', 'url': None, 'defaultConnectorId': None, 'defaultConnectorLabel': None, 'authenticationScheme': 'fields', 'connectorValidatorPresent': False, 'resourceBundlePresent': False, 'moduleHandler': None, 'iconPicker': False, 'unsafe': False, 'scope': '0', 'producer': 'DOMO', 'version': '0', 'dateCreated': 1601579548000, 'dateUpdated': 1693235941000, 'visibility': 'WITH_CONNECTOR', 'type': 'STANDARD', 'customerOAuthConfigurationDataProviderType': None, 'authenticationSchemeConfiguration': [{'id': 12114, 'name': 'authentication', 'type': 'dropdown', 'text': 'Authentication Type', 'tooltipText': 'Authentication Type', 'required': False, 'encrypted': False, 'defaultValue': 'basic', 'restrictions': None, 'sortOrde